In [58]:
 # Dependencies
import requests
import json
import pandas as pd

In [59]:
# Set the geographical coordinates for Brisbane, Australia
latitude = 40.7128
longitude = -74.0060

# Set the parameters for the type of place
categories = "accommodation"

# Set the parameters for the type of search
bias = f"proximity:{longitude},{latitude}"


# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":500,
    "bias":bias,
    "apiKey":'be13e77afa4d4c778e3673d5943b0949'    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"
response=requests.get(base_url, params=params)
response_data = response.json()

In [60]:
print(response.url)

https://api.geoapify.com/v2/places?categories=accommodation&limit=20&bias=proximity%3A-74.006%2C40.7128&apiKey=be13e77afa4d4c778e3673d5943b0949


In [40]:
# Print the json (pretty printed)
print(json.dumps(response_data, indent=4, sort_keys=True))



{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -74.00693649999998,
                    40.71103290035861
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "The Beekman, A Thompson Hotel",
                "address_line2": "123 Nassau Street, New York, NY 10038, United States of America",
                "categories": [
                    "accommodation",
                    "accommodation.hotel",
                    "internet_access"
                ],
                "city": "New York",
                "country": "United States",
                "country_code": "us",
                "county": "New York County",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "addr:city": "New York",
   

In [68]:
response_data['features'][0]

{'type': 'Feature',
 'properties': {'name': 'The Beekman, A Thompson Hotel',
  'country': 'United States',
  'country_code': 'us',
  'state': 'New York',
  'county': 'New York County',
  'city': 'New York',
  'postcode': '10038',
  'district': 'Manhattan Community Board 1',
  'suburb': 'Manhattan',
  'street': 'Nassau Street',
  'housenumber': '123',
  'lon': -74.0069365,
  'lat': 40.7110329,
  'state_code': 'NY',
  'formatted': 'The Beekman, A Thompson Hotel, 123 Nassau Street, New York, NY 10038, United States of America',
  'address_line1': 'The Beekman, A Thompson Hotel',
  'address_line2': '123 Nassau Street, New York, NY 10038, United States of America',
  'categories': ['accommodation', 'accommodation.hotel', 'internet_access'],
  'details': ['details', 'details.facilities'],
  'datasource': {'sourcename': 'openstreetmap',
   'attribution': '© OpenStreetMap contributors',
   'license': 'Open Database Licence',
   'url': 'https://www.openstreetmap.org/copyright',
   'raw': {'name

In [72]:
# Set the geographical coordinates for Brisbane, Australia
latitude = 40.7128
longitude = -74.0060

# Set the parameters for the type of place
categories = "catering"

# Set the parameters for the type of search
bias = f"proximity:{longitude},{latitude}"


# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":500,
    "bias":bias,
    "apiKey":'be13e77afa4d4c778e3673d5943b0949'    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"
response=requests.get(base_url, params=params)
response_cat = response.json()

In [73]:
# Print the json (pretty printed)
print(json.dumps(response_cat, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -74.0066898,
                    40.71164330035869
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Starbucks",
                "address_line2": "28 Park Row, New York, NY 10038, United States of America",
                "categories": [
                    "catering",
                    "catering.cafe",
                    "catering.cafe.coffee",
                    "catering.cafe.coffee_shop",
                    "internet_access"
                ],
                "city": "New York",
                "country": "United States",
                "country_code": "us",
                "county": "New York County",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
   

In [69]:
response_cat['features'][0]

{'type': 'Feature',
 'properties': {'name': 'Starbucks',
  'country': 'United States',
  'country_code': 'us',
  'state': 'New York',
  'county': 'New York County',
  'city': 'New York',
  'postcode': '10038',
  'district': 'Financial District',
  'suburb': 'Manhattan',
  'quarter': 'Lower Manhattan',
  'street': 'Park Row',
  'housenumber': '28',
  'lon': -74.0066898,
  'lat': 40.7116433,
  'state_code': 'NY',
  'formatted': 'Starbucks, 28 Park Row, New York, NY 10038, United States of America',
  'address_line1': 'Starbucks',
  'address_line2': '28 Park Row, New York, NY 10038, United States of America',
  'categories': ['catering',
   'catering.cafe',
   'catering.cafe.coffee',
   'catering.cafe.coffee_shop',
   'internet_access'],
  'details': ['details', 'details.catering', 'details.facilities'],
  'datasource': {'sourcename': 'openstreetmap',
   'attribution': '© OpenStreetMap contributors',
   'license': 'Open Database Licence',
   'url': 'https://www.openstreetmap.org/copyright

In [74]:
# Set the geographical coordinates for Brisbane, Australia
latitude = 40.7128
longitude = -74.0060

# Set the parameters for the type of place
categories = "entertainment"

# Set the parameters for the type of search
bias = f"proximity:{longitude},{latitude}"


# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":limit,
    "bias":bias,
    "apiKey":'be13e77afa4d4c778e3673d5943b0949'    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

In [75]:
# Run request
response_entertainment = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response_entertainment, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -74.01229239634087,
                    40.71265066401296
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Ronald O. Perelman Performing Arts Center",
                "address_line2": "58 Church Street, New York, NY 10048, United States of America",
                "categories": [
                    "building",
                    "building.entertainment",
                    "entertainment",
                    "entertainment.culture",
                    "entertainment.culture.theatre",
                    "wheelchair",
                    "wheelchair.yes"
                ],
                "city": "New York",
                "country": "United States",
                "country_code": "us",
                "county": "New York County",
                "datasource": {
                    "attribution